# Linking scRNA-seq data against `Gene`

In [ ]:
!lamin login testuser2

In [ ]:
import lamindb as ln
import lamindb.schema as lns
import bionty as bt

ln.nb.header()

Consider an scRNA-seq count matrix in form of an `AnnData` object in memory:

In [ ]:
adata = ln.dev.datasets.anndata_mouse_sc_lymph_node()

In [ ]:
adata

Check out the features of this dataset:

In [ ]:
adata.var.head()

## Configure a `Gene` reference for parsing features

The features in this data object are genes and indexed by Ensembl gene ids. We'd like to link these features so that we can query the data by genes!

Features are often knowledge-based entities. [Bionty](https://lamin.ai/docs/bionty) provides several knowledge-based tables for basic biological entities.

```{note}

- For an overview of knowledge tables, see: {mod}`~bionty`.
- For an overview of lookup identifiers, see: [Bionty lookup](https://lamin.ai/docs/bionty/bionty.lookup).
```

Let's first configure a gene reference we'd like to use for parsing features in this dataset:

In [ ]:
reference = bt.Gene(
    id=bt.lookup.gene_id.ensembl_gene_id,
    species=bt.Species().lookup.mouse,
)

## Parse features

Now let's parse the features from the data using the configured reference above:

In [ ]:
features = lns.Features(data=adata, ref=reference)

Here, all features were successfully (unambiguously) linked against their canonical reference in `bionty.Gene`.

This creates a feature set of type `gene` linked (indexed by its hash):

In [ ]:
features

This feature set links records for 10k genes. Here are the first 3, all of which can be queried:

In [ ]:
features.genes[:3]

Hence, not just for Ensemble IDs, but also by gene symbol, NCBI ids, gene type, etc.

## Track data with features (genes)

Now we can track data together with the parsed features by passing `features` when instantiating `DObject`.

```{tip}

Linking features can also be made post instantiation via:
```python

dobject = ln.DObject(adata, name="Mouse Lymph Node scRNA-seq")
dobject.features = features
```

In [ ]:
dobject = ln.DObject(adata, name="Mouse Lymph Node scRNA-seq", features=features)

The features can now be accessed via relationship to dobejct:

In [ ]:
dobject.features[0].genes[:3]

Add dobject and its linked features to the database:

In [ ]:
ln.add(dobject)

## Querying data by features

```{seealso}

Basic queries:

- {doc}`/guide/select`
- {doc}`/guide/query-book`

```

Let us query gene records by symbol:

In [ ]:
ln.select(lns.bionty.Gene, symbol="Actg1").df()

How do we retrieve data objects, in which this gene was read out? 

In [ ]:
ln.select(ln.DObject).df()

We could first query all feature sets that contain the gene, and then query `DObject` by that.

In [ ]:
features = (
    ln.select(lns.Features, lns.bionty.Gene.symbol)
    .join(lns.Features.genes)
    .where(lns.bionty.Gene.symbol == "Actg1")
)
features.df().head()

Of course, we can also write everything in one statement:

In [ ]:
dobjects = (
    ln.select(ln.DObject)
    .join(ln.DObject.features)
    .join(lns.Features.genes)
    .where(lns.bionty.Gene.symbol == "Actg1")
)
dobjects.df().head()